# Auto Encoder

In [ ]:
from __future__ import print_function

%config IPCompleter.greedy=True

import argparse, torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt

batchsize = 128
epochs = 10
loginterval = 10
cuda = torch.cuda.is_available()

SEED = 1
torch.manual_seed(SEED)

device = torch.device("cuda" if cuda else "cpu")

kwargs = {'num_workers': 4, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/tmp/data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batchsize, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/tmp/data', train=False, transform=transforms.ToTensor()),
    batch_size=batchsize, shuffle=True, **kwargs)

In [ ]:
class AE(nn.Module):
    """
    Auto Encoder model, with one hidden layer.
    
    Attributes
    ----------
    encoder : torch.nn.modules.linear.Sequential
        The first fully-connected layer.
    decoder : torch.nn.modules.linear.Sequential
        The second fully connected layer.
    
    Examples
    --------
    >>> model = AE()
    >>> if cuda:
    ...     model.cuda()
    >>>> optimizer = optim.Adam(model.parameters(), lr=1e-3)
    """
    
    def __init__(self):
        """
        Creation of the model.
        """
        super(AE, self).__init__()

        # encoder :FC (784 -> 400), RELU activation; FC (400, 20)
        self.encoder = nn.Sequential(nn.Linear(784, 400), nn.ReLU(), nn.Linear(400, 20))
        # decoder :FC (20 -> 400), RELU activation; FC (400, 784)
        self.decoder = nn.Sequential(nn.Linear(20, 400), nn.ReLU(), nn.Linear(400, 784))

    def encode(self, x):
        """
        The encode pass.
        
        Parameters
        ----------
        x : tensor_like
            The input of the layer.
        
        Returns
        -------
        out : tensor_like
            The output of the layer.
        """
        return self.encoder(x)

    def decode(self, z):
        """
        The decode pass.
        
        Parameters
        ----------
        x : tensor_like
            The input of the layer.
        
        Returns
        -------
        out : tensor_like
            The output of the layer.
        """
        return torch.sigmoid(self.decoder(z))

    def forward(self, x):
        """
        The forward pass of the global network.
        
        Parameters
        ----------
        x : tensor_like
            The input of the network.
        
        Returns
        -------
        out : tensor_like
            The output of the network.
        """
        xview = x.view((-1, 784))
        z = self.encode(xview)
        return self.decode(z)


model = AE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def loss_function(recon_batch, x):
    """
    Compute the bce as the binary-cross entropy across the batch as a sum.
    
    Parameters
    ----------
    recon_batch : tensor_like
        The output of the network.
    x : tensor_like
        The desired output.
    
    Returns
    -------
    The binary-cross entropy of the state.
    """
    bce = nn.BCELoss(reduction='sum')
    return bce(recon_batch, x)

In [ ]:
losses = []

def train(epoch):
    """
    The train phase of the model.
    
    Parameters
    ----------
    epoch : int
        The number of epoch to perform.
    """
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch = model(data)
        loss = loss_function(recon_batch, data.view(-1, 784))
        losses.append(loss.cpu().item())
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % loginterval == 0:
            print('\r Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.1f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)), end='')

    print(' Average loss: {:.1f}'.format(train_loss / len(train_loader.dataset)), end='')

In [ ]:
def test(epoch):
    """
    The test phase of the model.
    
    Parameters
    ----------
    epoch : int
        The number of epoch to perform.
    """
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch = model(data)
            test_loss += loss_function(recon_batch, data.view(-1, 784))

            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batchsize, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print(' Test loss: {:.1f}'.format(test_loss))

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
     sample = torch.randn(64, 20).to(device)
     sample = model.decode(sample).cpu()
     save_image(sample.view(64, 1, 28, 28), 'results/sample_' + str(epoch) + '.png')

In [ ]:
plt.plot(losses)